In [ ]:
# 모듈 임포트
import time
import board
import busio
import adafruit_ads1x15.ads1115 as ADS
from adafruit_ads1x15.analog_in import AnalogIn
import joblib
import numpy as np
import psutil
import pandas as pd
from sklearn.preprocessing import StandardScaler
import csv
from datetime import datetime

In [ ]:
# 모델 및 스케일러 로드
model = joblib.load('./model/random_forest_model.joblib')  # ← 변경된 모델 파일
scaler = joblib.load('./model/scaler_rf.joblib')


In [ ]:
# I2C 및 ADC 설정
i2c = busio.I2C(board.SCL, board.SDA)
ads = ADS.ADS1115(i2c) # (아날로그 → 디지털 변환기) 객체
ads.gain = 1 # 증폭 배율 설정

# ADS1115 채널 선택
voltage_ch = AnalogIn(ads, ADS.P0)
current_ch = AnalogIn(ads, ADS.P1)

In [ ]:
# 센서 보정값
ZMPT_offset = 2.5539
ZMPT_scale = 997.6

ACS712_offset = 2.5087
ACS712_sensitivity = 0.0990

In [ ]:
# 파생 피처 생성 함수
def add_features(voltage_seq, current_seq):
    voltage = np.mean(voltage_seq)
    current = np.mean(current_seq)
    voltage_diff = np.abs(voltage_seq[-1] - voltage_seq[-2])
    current_diff = np.abs(current_seq[-1] - current_seq[-2])
    voltage_ma = np.mean(voltage_seq[-5:])
    current_ma = np.mean(current_seq[-5:])
    power = voltage * current
    power_diff = np.abs(power - (voltage_seq[-2] * current_seq[-2]))
    return [voltage, current, voltage_diff, current_diff, voltage_ma, current_ma, power, power_diff]

In [ ]:
def postprocess_anomalies_realtime(anomaly_buffer, min_consecutive=8):
    if len(anomaly_buffer) < min_consecutive:
        return False
    
    # 정확히 연속된 1이 min_consecutive 이상인지 확인
    count = 0
    for val in reversed(anomaly_buffer): # 최근 값부터 역순으로 확인
        if val == 1:
            count += 1
        else:
            break
    
    return count >= min_consecutive

In [ ]:
# 테스트 데이터 불러오기 (센서 대체용 - 사용 안 할 경우 생략 가능)
column_names = ['timestamp', 'v_raw', 'c_raw', 'voltage', 'current']
df = pd.read_csv('./log/raw_dataset_arc.csv', names=column_names, header=None)
df = df.dropna(subset=['voltage', 'current']).reset_index(drop=True)
csv_filename = "./log/rf_log5.csv"

In [ ]:
try:
    print("🚀 실시간 아크 감지 시작합니다!")

    USE_SENSOR = True
    ANOMALY_THRESHOLD = 2

    data_buffer = []
    anomaly_buffer = []
    first_anomaly_detected = False
    anomaly_start_time = None
    start_time = time.perf_counter()
    idx = -1

    while True:
        idx += 1

        if USE_SENSOR:
            try:
                v_raw_sensor = voltage_ch.voltage
                c_raw_sensor = current_ch.voltage
                voltage = (v_raw_sensor - ZMPT_offset) * ZMPT_scale
                current = (c_raw_sensor - ACS712_offset) / ACS712_sensitivity

                prev_voltage = voltage
                prev_current = current

            except (OSError, ValueError, RuntimeError) as e:
                print(f"⚠️ 센서 오류 발생: {e}")
                voltage = prev_voltage if 'prev_voltage' in locals() else 0
                current = prev_current if 'prev_current' in locals() else 0

        else:
            voltage = df['voltage'].iloc[idx]
            current = df['current'].iloc[idx]

        data_buffer.append((voltage, current))
        anomaly = 0
        if len(data_buffer) >= 8:
            voltage_seq, current_seq = zip(*data_buffer[-8:])
            features = add_features(voltage_seq, current_seq)
            features_scaled = scaler.transform([features])
            anomaly = int(model.predict(features_scaled).flatten()[0])  # RandomForest는 0/1 반환

            anomaly_buffer.append(anomaly)
            if len(anomaly_buffer) > ANOMALY_THRESHOLD:
                anomaly_buffer.pop(0)

            if anomaly == 1 and anomaly_start_time is None:
                anomaly_start_time = time.perf_counter()
            elif anomaly == 0:
                anomaly_start_time = None

            if postprocess_anomalies_realtime(anomaly_buffer, min_consecutive=ANOMALY_THRESHOLD):
                print("⚡ 아크 이상 감지!", idx)
                if not first_anomaly_detected and anomaly_start_time is not None:
                    total_elapsed = time.perf_counter() - start_time
                    actual_elapsed = time.perf_counter() - anomaly_start_time
                    print(f"⏱️ 전체 소요 시간: {total_elapsed:.6f}초")
                    print(f"⏱️ 연속 2개 이상 신호까지 실제 소요 시간: {actual_elapsed:.6f}초")
                    first_anomaly_detected = True
                            # CSV 기록
                    with open(csv_filename, mode='a', newline='') as file:
                        writer = csv.writer(file)
                        timestamp = datetime.now().isoformat()
                        writer.writerow([timestamp, voltage, current, anomaly])
                    break

        # CSV 기록
        with open(csv_filename, mode='a', newline='') as file:
            writer = csv.writer(file)
            timestamp = datetime.now().isoformat()
            writer.writerow([timestamp, voltage, current, anomaly])

        # 메모리 및 CPU 사용량 출력
        process = psutil.Process()
        cpu_usage = process.cpu_percent(interval=0.0)
        # cpu_usage = psutil.cpu_percent(interval=0.0)
        memory_usage = psutil.Process().memory_info().rss / 1024 ** 2
        print(f"🧠 메모리: {memory_usage:.2f}MB | 🧮 CPU: {cpu_usage:.2f}% | idx: {idx}")
        print(f"v_raw:{v_raw_sensor}, c_raw:{c_raw_sensor}, voltage:{voltage}, current:{current}")

        time.sleep(0.001163)  # 860Hz

except KeyboardInterrupt:
    print("🛑 실시간 감지를 종료합니다.")


🚀 실시간 아크 감지 시작합니다!
🧠 메모리: 248.46MB | 🧮 CPU: 7.30% | idx: 0
v_raw:2.5343750000000003, c_raw:2.54975, voltage:-19.478139999999794, current:0.4146464646464627
🧠 메모리: 248.46MB | 🧮 CPU: 22.20% | idx: 1
v_raw:2.543, c_raw:2.549875, voltage:-10.87383999999991, current:0.4159090909090905
🧠 메모리: 248.46MB | 🧮 CPU: 0.00% | idx: 2
v_raw:2.56275, c_raw:2.5496250000000003, voltage:8.828759999999804, current:0.41338383838383935
🧠 메모리: 248.46MB | 🧮 CPU: 0.00% | idx: 3
v_raw:2.560375, c_raw:2.5496250000000003, voltage:6.459460000000009, current:0.41338383838383935
🧠 메모리: 248.46MB | 🧮 CPU: 0.00% | idx: 4
v_raw:2.549, c_raw:2.550375, voltage:-4.888240000000127, current:0.42095959595959276
🧠 메모리: 248.46MB | 🧮 CPU: 11.10% | idx: 5
v_raw:2.555, c_raw:2.5505, voltage:1.0973600000001007, current:0.42222222222222056
🧠 메모리: 248.46MB | 🧮 CPU: 0.00% | idx: 6
v_raw:2.557, c_raw:2.5500000000000003, voltage:3.092559999999881, current:0.4171717171717183
🧠 메모리: 248.46MB | 🧮 CPU: 15.80% | idx: 7
v_raw:2.547625, c_raw:2